In [9]:
import pandas as pd
import numpy as np
from transformers import *

import matplotlib.pyplot as plt

In [3]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
train.replace(r'^\s*$', np.nan, regex=True, inplace=True)

In [4]:
train.dropna(inplace=True)

In [5]:
total_missing = train.isnull().sum().sort_values(ascending = False)
total_missing

sentiment        0
selected_text    0
text             0
textID           0
dtype: int64

In [10]:
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [7]:
test.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [25]:
train.groupby('sentiment')['textID'].nunique()

sentiment
negative     7781
neutral     11117
positive     8582
Name: textID, dtype: int64

In [42]:
def generate_tokenizer():
    tokenizer_list = []
    MODELS = [(BertTokenizer, 'bert-base-cased'), (RobertaTokenizer, 'roberta-base')]
    for tokenizer_class, weights in MODELS:
        tokenizer = tokenizer_class.from_pretrained(weights)
        tokenizer_list.append(tokenizer)
    return tokenizer_list

tokenizers = generate_tokenizer()

def tokenize(text):
    for tokenizer in tokenizers:
        return tokenizer.encode(text, add_special_tokens=True, padding='max_length', max_length=282)

train['tokenized_text'] = train.apply(lambda x: tokenize(x['text']), axis=1)
train['tokenized_selected_text'] = train.apply(lambda x: tokenize(x['selected_text']), axis=1)

print('max tokenized_text size: ', max(len(s) for s in train['tokenized_text']))
print('max tokenized_selected_text size: ', max(len(s) for s in train['tokenized_selected_text']))

max tokenized_text size:  282
max tokenized_selected_text size:  282


In [19]:
word2idx = []
for tokenizer in tokenizers:
    word2idx.append(tokenizer.word_to_tokens)

Using bos_token, but it is not set yet.


In [44]:
import flair

In [53]:
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')
s = flair.data.Word('you')
flair_sentiment.predict(s)
total_sentiment = s.labels
total_sentiment

2020-07-04 20:06:52,843 loading file /Users/pattersonwu/.flair/models/sentiment-en-mix-distillbert.pt


AttributeError: module 'flair.data' has no attribute 'Word'